In [ ]:
import os
import numpy as np
import cv2
from pathlib import Path
import matplotlib.pyplot as plt
from tqdm import tqdm

import gtsam
from gtsam import imuBias, noiseModel, PriorFactorConstantBias
from gtsam.symbol_shorthand import B, V, X, L

from lac.perception.depth import project_pixel_to_rover
from lac.utils.frames import apply_transform
from lac.utils.plotting import plot_3d_points, plot_surface, plot_poses, plot_path_3d
from lac.util import load_data
from lac.params import LAC_BASE_PATH, DT

%load_ext autoreload
%autoreload 2

In [ ]:
data_path = Path(LAC_BASE_PATH) / "output/DataCollectionAgent/stereo_lights1.0_map1_preset0"
initial_pose, lander_pose, poses, imu_data, cam_config = load_data(data_path)
print(f"Num poses: {len(poses)}")

In [ ]:
g = 1.622
IMU_PARAMS = gtsam.PreintegrationParams.MakeSharedU(-g)
gyro_sigma = 1e-3
accel_sigma = 1e-3
I_3x3 = np.eye(3)
IMU_PARAMS.setGyroscopeCovariance(gyro_sigma**2 * I_3x3)
IMU_PARAMS.setAccelerometerCovariance(accel_sigma**2 * I_3x3)
IMU_PARAMS.setIntegrationCovariance(1e-7**2 * I_3x3)

In [ ]:
graph = gtsam.NonlinearFactorGraph()
initial_estimate = gtsam.Values()

# Fix first pose
graph.add(gtsam.NonlinearEqualityPose3(X(0), gtsam.Pose3(poses[0])))
initial_estimate.insert(X(0), gtsam.Pose3(poses[0]))

# Zero bias
BIAS_KEY = B(0)
zero_bias = imuBias.ConstantBias(np.zeros(3), np.zeros(3))
# graph.add(PriorFactorConstantBias(BIAS_KEY, zero_bias, noiseModel.Constrained.All(6)))
BIAS_NOISE = gtsam.noiseModel.Isotropic.Sigma(6, 1e-2)
graph.add(PriorFactorConstantBias(BIAS_KEY, zero_bias, BIAS_NOISE))
initial_estimate.insert(BIAS_KEY, zero_bias)

# Zero initial velocity prior
VELOCITY_NOISE = gtsam.noiseModel.Isotropic.Sigma(3, 1e-2)
graph.push_back(gtsam.PriorFactorVector(V(0), np.zeros(3), VELOCITY_NOISE))
initial_estimate.insert(V(0), np.zeros(3))

# Preintegrator
accum = gtsam.PreintegratedImuMeasurements(IMU_PARAMS)

N_FRAMES = 1000
RATE = 10

i = 0

for k in range(1, N_FRAMES):
    accel = imu_data[k, :3]
    gyro = imu_data[k, 3:]
    accum.integrateMeasurement(accel, gyro, DT)

    if k % RATE == 0:
        i += 1
        initial_estimate.insert(X(i), gtsam.Pose3(poses[k]))

        initial_estimate.insert(V(i), np.array([0.0, 0.0, 0]))
        # initial_velocity = (poses[i][:3,3] - poses[i-1][:3,3]) / DT
        # initial_estimate.insert(V(i), initial_velocity)

        graph.add(gtsam.ImuFactor(X(i - 1), V(i - 1), X(i), V(i), BIAS_KEY, accum))

        accum.resetIntegration()

In [ ]:
params = gtsam.LevenbergMarquardtParams()

optimizer = gtsam.LevenbergMarquardtOptimizer(graph, initial_estimate, params)
result = optimizer.optimize()

In [ ]:
# Get bias
bias = result.atConstantBias(BIAS_KEY)
print(f"Bias: {bias}")

In [ ]:
est_traj = np.array([result.atPose3(X(k)).translation() for k in range(i)])

fig = plot_poses(poses[:N_FRAMES], no_axes=True, color="black", name="Ground Truth")
fig = plot_path_3d(est_traj, fig=fig, color="red", name="GTSAM IMU")
fig.show()